In [1]:
using Pkg;
Pkg.activate("..");
Pkg.instantiate();

  Activating project at `~/dev/uni/amo-individual-project`


In [2]:
using WebSockets

In [3]:
import Sockets
const LOCALIP = string(Sockets.getipaddr())
const PORT = 1234

function coroutine(ws)
    @info "Started coroutine for " ws
    while isopen(ws)
        data, = readguarded(ws)
        s = String(data)
        if s == ""
            writeguarded(ws, "Goodbye!")
            break
        end
        @info "Received: $s"
        writeguarded(ws, "Hello! Send empty message to exit, or just leave.")
    end
    @info "Will now close " ws
end

function gatekeeper(req, ws)
    orig = WebSockets.origin(req)
    @info "\nOrigin: $orig   Target: $(req.target)   subprotocol: $(subprotocol(req))"
    if occursin(LOCALIP, orig)
        coroutine(ws)
    elseif orig == ""
        @info "Non-browser clients don't send Origin. We liberally accept the update request in this case:" ws
        coroutine(ws)
    else
        @warn "Inacceptable request"
    end
end

handle(req) = println("Got request: ", req.target)

const server = WebSockets.ServerWS(handle, gatekeeper)

@info "In browser > $LOCALIP:$PORT , F12> console > ws = new WebSocket(\"ws://$LOCALIP:$PORT\") "
WebSockets.with_logger(WebSocketLogger()) do
    WebSockets.serve(server, LOCALIP, PORT)
end

┌ Info: In browser > 192.168.0.129:1234 , F12> console > ws = new WebSocket("ws://192.168.0.129:1234") 
└ @ Main /home/moritz/dev/uni/amo-individual-project/notebooks/ws.ipynb:37


┌ Info: 
└ Origin:    Target: /   subprotocol: 
┌ Info: Non-browser clients don't send Origin. We liberally accept the update request in this case:
└   ws = WebSocket{TCPSocket}(server, CONNECTED): ✓


┌ Info: Started coroutine for 
└   ws = WebSocket{TCPSocket}(server, CONNECTED): ✓
[ Info: Received: Hello from client 1
┌ Info: Will now close 
└   ws = WebSocket{TCPSocket}(server, CONNECTED): paused, 6 bytes
